In [65]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Conv2D, Dense, Dropout

In [66]:
cifar10 = tf.keras.datasets.cifar10
(X_train,Y_train),(X_test,Y_test) = cifar10.load_data()

In [67]:
X_train[0].shape

(32, 32, 3)

In [68]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [69]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,Y_train)).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test,Y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [70]:
class MyCifar10Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(64, (4,4), strides=2, padding='same', activation="relu")
        self.conv2 = Conv2D(128, (4,4), strides=2, padding='same', activation="relu")
        self.conv3 = Conv2D(256, (4,4), strides=2, padding='same', activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(64, activation="relu")
        self.d2 = Dense(32, activation="relu")
        self.drop = Dropout(0.3)
        self.d3 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.drop(x)
        x = self.conv2(x)
        x = self.drop(x)
        x = self.conv3(x)
        x = self.drop(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.drop(x)
        x = self.d3(x)
        return x

model = MyCifar10Model()

In [71]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [72]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [73]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [74]:
EPOCHS = 6

for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for images, labels in train_dataset:
      train_step(images, labels)

    for test_images, test_labels in test_dataset:
      test_step(test_images, test_labels)

    print(
      f'Epoch {epoch + 1}, '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result()}, '
      f'Test Loss: {test_loss.result()}, '
      f'Test Accuracy: {test_accuracy.result()}'
    )

Epoch 1, Loss: 1.6196354627609253, Accuracy: 0.4081000089645386, Test Loss: 1.3336308002471924, Test Accuracy: 0.5212000012397766
Epoch 2, Loss: 1.1773052215576172, Accuracy: 0.5808600187301636, Test Loss: 1.0742518901824951, Test Accuracy: 0.6234999895095825
Epoch 3, Loss: 0.9571167826652527, Accuracy: 0.6629599928855896, Test Loss: 0.9961808323860168, Test Accuracy: 0.6485999822616577
Epoch 4, Loss: 0.7879047989845276, Accuracy: 0.7209600210189819, Test Loss: 0.9947006702423096, Test Accuracy: 0.6694999933242798
Epoch 5, Loss: 0.6381117701530457, Accuracy: 0.7763800024986267, Test Loss: 1.108596682548523, Test Accuracy: 0.6446999907493591
Epoch 6, Loss: 0.5034162402153015, Accuracy: 0.8230999708175659, Test Loss: 1.2016844749450684, Test Accuracy: 0.66839998960495
